In [1]:
import os
import pandas as pd
import numpy as np
import json as json
import csv

In [3]:
##### CONSERVATION DATA #####

# Import data
CONSERVATION_DATA = pd.read_excel('uw_supplier_data110116.xlsx')

# Extract columns of interest
conservation_data = pd.DataFrame(CONSERVATION_DATA.iloc[1:,[0,3,12,16,18,19,20,21,22,24,25,26]])
conservation_data.columns = [
    'supplier_name',
    'date',
    'population',
    'conservation_standard',
    'production_month', # (gal) Total monthly potable water production, calculated by Water Board
    'production_2013', # (gal) Total monthly potable water production in 2013, calculated by Water Board
    'CII_production', # (gal) Subset of monthly potable water production for CII, calculated by Water Board
    'gpcd', # R_GPCD calculated by SWRCB based on supplier data
    'residential_use', # percentage
    'hydrologic_region',
    'water_days', # days per week allowed
    'complaints', # complaints received
]

# Clean data:
# No conservation standards in summer 2014 --> replace Nan with 0
conservation_data['conservation_standard'].fillna(0, inplace=True)
# Assume NaN in watering days indicate no policy limiting irrigation (e.g. 2014), or different policy approaches
conservation_data['water_days'].fillna(7, inplace=True)
# Replace Null CII water use with 0
conservation_data['CII_production'].replace('Null', 0, inplace=True)

# Calculate water production for residential water use each month (gal)
conservation_data['res_production'] = conservation_data['gpcd']*conservation_data['population']*31
# Calculate conservation ammount (gal wrt same month in 2013)
conservation_data['conservation_month_gal'] = conservation_data['production_2013'] - conservation_data['production_month']

# Convert month column to datetime format
conservation_data['date'] = pd.to_datetime(conservation_data['date'])
# Add year column
conservation_data['year'] = pd.DatetimeIndex(conservation_data['date']).year
# Add month column
conservation_data['month'] = pd.DatetimeIndex(conservation_data['date']).month
# Convert month column back to object
conservation_data['date'] = conservation_data['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

# Display first rows of table
conservation_data.head()

,supplier_name,date,population,conservation_standard,production_month,production_2013,CII_production,gpcd,residential_use,hydrologic_region,water_days,complaints,res_production,conservation_month_gal,year,month
1,East Bay Municipal Utilities District,2016-08-15,1400000,0.00,6.007500e+09,7.172300e+09,1.141000e+09,84.437212,61.0,San Francisco Bay,7.0,176,3.664575e+09,1.164800e+09,2016,8
2,East Bay Municipal Utilities District,2016-07-15,1400000,0.00,6.056600e+09,7.452200e+09,9.940000e+08,83.731797,60.0,San Francisco Bay,7.0,197,3.633960e+09,1.395600e+09,2016,7
3,East Bay Municipal Utilities District,2016-06-15,1400000,0.00,5.675900e+09,6.927500e+09,8.390000e+08,82.435690,61.0,San Francisco Bay,2.0,188,3.577709e+09,1.251600e+09,2016,6
4,East Bay Municipal Utilities District,2016-05-15,1400000,0.16,4.959300e+09,6.716500e+09,9.550000e+08,68.561751,60.0,San Francisco Bay,2.0,166,2.975580e+09,1.757200e+09,2016,5
5,East Bay Municipal Utilities District,2016-04-15,1400000,0.16,4.018800e+09,5.417500e+09,8.300000e+08,59.325143,62.0,San Francisco Bay,2.0,166,2.574711e+09,1.398700e+09,2016,4


In [4]:
conservation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11348 entries, 1 to 11348
Data columns (total 16 columns):
supplier_name             11348 non-null object
date                      11348 non-null object
population                11348 non-null int64
conservation_standard     11348 non-null float64
production_month          11348 non-null float64
production_2013           11348 non-null float64
CII_production            11348 non-null float64
gpcd                      11348 non-null float64
residential_use           11348 non-null float64
hydrologic_region         11348 non-null object
water_days                11348 non-null float64
complaints                11348 non-null int64
res_production            11348 non-null float64
conservation_month_gal    11348 non-null float64
year                      11348 non-null int32
month                     11348 non-null int32
dtypes: float64(9), int32(2), int64(2), object(3)
memory usage: 1.3+ MB


In [6]:
# Save as csv
conservation_data.to_csv('conservation.csv', index=False)

In [5]:
##### CONSERVATION June 2015 - May 2016 #####

# Extract rows of interest (June 2015 - May 2016)
months_2015 = ['2015-06-15', '2015-07-15', '2015-08-15', '2015-09-15', '2015-10-15', '2015-11-15', '2015-12-15',
              '2016-01-15', '2016-02-15', '2016-03-15', '2016-04-15', '2016-05-15']
conservation_2015 = conservation_data.copy()
conservation_2015 = conservation_2015[conservation_2015['date'].isin(months_2015)]

# Sum conservation for the whole year
conservation = pd.DataFrame({'conservation' : conservation_2015.groupby('supplier_name').sum()['conservation_month_gal']}).reset_index()
# Sum residential water use for the whole year
conservation_2015['residential'] = conservation_2015['residential_use']/100*conservation_2015['production_month']
residential = pd.DataFrame({'residential' : conservation_2015.groupby('supplier_name').sum()['residential']}).reset_index()
# Sum total water use in 2013
production_2013 = pd.DataFrame({'production_2013' : conservation_2015.groupby('supplier_name').sum()['production_2013']}).reset_index()
# Extract other parameters that stay the same
latest = ['2016-05-15']
latest_data = conservation_data.copy()
latest_data = latest_data[latest_data['date'].isin(latest)]
general_data = latest_data.loc[1:,['supplier_name', 'population', 'hydrologic_region', 'conservation_standard']]

# Merge data frames
conservation_2015_agg = pd.merge(conservation, residential, on='supplier_name', how='outer')
conservation_2015_agg = pd.merge(conservation_2015_agg, production_2013, on='supplier_name', how='outer')
conservation_2015_agg = pd.merge(conservation_2015_agg, general_data, on='supplier_name', how='outer')

# Compute other values of interest
conservation_2015_agg['percent'] = conservation_2015_agg['conservation']/conservation_2015_agg['production_2013']

conservation_2015_agg.head()

,supplier_name,conservation,residential,production_2013,population,hydrologic_region,conservation_standard,percent
0,Adelanto City of,2.184056e+08,9.352085e+08,1.509218e+09,28842,South Lahontan,0.16,0.144714
1,Alameda County Water District,4.615207e+09,8.137271e+09,1.624100e+10,348000,San Francisco Bay,0.16,0.284170
2,Alco Water Service,3.553696e+08,9.894673e+08,1.495806e+09,29568,Central Coast,0.24,0.237577
3,Alhambra City of,8.631177e+08,2.725762e+09,3.676360e+09,85068,South Coast,0.22,0.234775
4,Amador Water Agency,3.512090e+08,4.953215e+08,1.211310e+09,23347,San Joaquin River,0.21,0.289941


In [11]:
conservation_2015_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408 entries, 0 to 407
Data columns (total 8 columns):
supplier_name            408 non-null object
conservation             408 non-null float64
residential              408 non-null float64
production_2013          408 non-null float64
population               408 non-null int64
hydrologic_region        408 non-null object
conservation_standard    408 non-null float64
percent                  408 non-null float64
dtypes: float64(5), int64(1), object(2)
memory usage: 28.7+ KB


In [12]:
# Save to csv
conservation_2015_agg.to_csv('conservation_2015.csv', index=False)

In [6]:
##### STATEWIDE WATER USE PER CAPITA 2013-2015 #####

# Extract rows of interest (June - May)
months_2014 = ['2014-06-15', '2014-07-15', '2014-08-15', '2014-09-15', '2014-10-15', '2014-11-15', '2014-12-15',
              '2015-01-15', '2015-02-15', '2015-03-15', '2015-04-15', '2015-05-15']
months_2015 = ['2015-06-15', '2015-07-15', '2015-08-15', '2015-09-15', '2015-10-15', '2015-11-15', '2015-12-15',
              '2016-01-15', '2016-02-15', '2016-03-15', '2016-04-15', '2016-05-15']
statewide_data = conservation_data.copy()
statewide_2014 = statewide_data[statewide_data['date'].isin(months_2014)]
statewide_2015 = statewide_data[statewide_data['date'].isin(months_2015)]

# Sum water production for the whole year
production_2015 = pd.DataFrame({'production_2015' : statewide_2015.groupby('supplier_name').sum()['production_month']}).reset_index()
production_2014 = pd.DataFrame({'production_2014' : statewide_2014.groupby('supplier_name').sum()['production_month']}).reset_index()
production_2013 = pd.DataFrame({'production_2013' : statewide_2015.groupby('supplier_name').sum()['production_2013']}).reset_index()
# Sum residential water production for the whole year
residential_2015 = pd.DataFrame({'residential_2015' : statewide_2015.groupby('supplier_name').sum()['res_production']}).reset_index()
residential_2014 = pd.DataFrame({'residential_2014' : statewide_2014.groupby('supplier_name').sum()['res_production']}).reset_index()

# Extract other parameters that stay the same
latest = ['2016-05-15']
latest_data = conservation_data.copy()
latest_data = latest_data[latest_data['date'].isin(latest)]
general_data = latest_data.loc[1:,['supplier_name', 'population', 'hydrologic_region']]

# Merge data frames
statewide_agg = pd.merge(production_2015, production_2014, on='supplier_name', how='outer')
statewide_agg = pd.merge(statewide_agg, production_2013, on='supplier_name', how='outer')
statewide_agg = pd.merge(statewide_agg, residential_2015, on='supplier_name', how='outer')
statewide_agg = pd.merge(statewide_agg, residential_2014, on='supplier_name', how='outer')
statewide_agg = pd.merge(statewide_agg, general_data, on='supplier_name', how='outer')

# Convert values to water use per capita
statewide_agg['ggpcd2015'] = statewide_agg['production_2015']/statewide_agg['population']/365
statewide_agg['ggpcd2015'] = statewide_agg['ggpcd2015'].round(2)
statewide_agg['ggpcd2014'] = statewide_agg['production_2014']/statewide_agg['population']/365
statewide_agg['ggpcd2014'] = statewide_agg['ggpcd2014'].round(2)
statewide_agg['ggpcd2013'] = statewide_agg['production_2014']/statewide_agg['population']/365
statewide_agg['ggpcd2013'] = statewide_agg['ggpcd2013'].round(2)
statewide_agg['rgpcd2015'] = statewide_agg['residential_2015']/statewide_agg['population']/365
statewide_agg['rgpcd2015'] = statewide_agg['rgpcd2015'].round(2)
statewide_agg['rgpcd2014'] = statewide_agg['residential_2015']/statewide_agg['population']/365
statewide_agg['rgpcd2014'] = statewide_agg['rgpcd2014'].round(2)

statewide_agg = statewide_agg.loc[1:,['supplier_name', 'production_2015', 'production_2014', 'production_2013', 'r2015', 'rgpcd2014', 'population', 'hydrologic_region']]

statewide_agg.head()
# save to csv
#statewide_agg.to_csv('statewide_agg.csv', index=False)

,supplier_name,production_2015,production_2014,production_2013,r2015,rgpcd2014,population,hydrologic_region
1,Alameda County Water District,1.162579e+10,1.139060e+10,1.624100e+10,NaN,65.25,348000,San Francisco Bay
2,Alco Water Service,1.140436e+09,1.339400e+09,1.495806e+09,NaN,93.37,29568,Central Coast
3,Alhambra City of,2.813242e+09,3.319690e+09,3.676360e+09,NaN,89.40,85068,South Coast
4,Amador Water Agency,8.601010e+08,1.009200e+09,1.211310e+09,NaN,59.16,23347,San Joaquin River
5,"American Canyon, City of",8.765628e+08,1.046961e+09,1.203043e+09,NaN,63.78,20315,San Francisco Bay


In [7]:
##### STATEWIDE WATER USE PER CAPITA 2013-2015 (MONTHLY) #####

statewide_monthly = conservation_data.copy()
statewide_monthly = statewide_monthly.loc[1:,['date','hydrologic_region','production_month','production_2013', 'population', 'residential_use']]
statewide_monthly['residential'] = statewide_monthly['production_month']*statewide_monthly['residential_use']/100
statewide_monthly['residential2013'] = statewide_monthly['production_2013']*statewide_monthly['residential_use']/100
#latest = ['2016-05-15']
#latest_data = conservation_data.copy()
#latest_data = latest_data[latest_data['date'].isin(latest)]
#population = pd.DataFrame({'population' : latest_data.groupby('hydrologic_region').sum()['population']}).reset_index()

population = pd.DataFrame({'population' : statewide_monthly.groupby('date').sum()['population']}).reset_index()
production = pd.DataFrame({'production' : statewide_monthly.groupby(['date']).sum()['production_month']}).reset_index()
production2013 = pd.DataFrame({'production2013' : statewide_monthly.groupby(['date']).sum()['production_2013']}).reset_index()
production_res = pd.DataFrame({'production_res': statewide_monthly.groupby(['date']).sum()['residential']}).reset_index()
production2013_res = pd.DataFrame({'production2013_res': statewide_monthly.groupby(['date']).sum()['residential2013']}).reset_index()

statewide_monthly_agg = pd.merge(production, production2013, on=['date'], how='outer')
statewide_monthly_agg = pd.merge(statewide_monthly_agg, production_res, on=['date'], how='outer')
statewide_monthly_agg = pd.merge(statewide_monthly_agg, production2013_res, on=['date'], how='outer')
statewide_monthly_agg = pd.merge(statewide_monthly_agg, population, on=['date'], how='outer')
statewide_monthly_agg['gpcd'] = statewide_monthly_agg['production']/statewide_monthly_agg['population']/30
statewide_monthly_agg['rgpcd'] = statewide_monthly_agg['production_res']/statewide_monthly_agg['population']/30
statewide_monthly_agg['gpcd2013'] = statewide_monthly_agg['production2013']/statewide_monthly_agg['population']/30
statewide_monthly_agg['rgpcd2013'] = statewide_monthly_agg['production2013_res']/statewide_monthly_agg['population']/30

statewide_monthly_agg.head()

,date,production,production2013,production_res,production2013_res,population,gpcd,rgpcd,gpcd2013,rgpcd2013
0,2014-06-15,2.062988e+11,2.158438e+11,1.368721e+11,1.432697e+11,34448471,199.620627,132.441381,208.856683,138.631934
1,2014-07-15,2.222755e+11,2.402838e+11,1.462707e+11,1.583496e+11,35576282,208.261858,137.048920,225.134807,148.366237
2,2014-08-15,2.080143e+11,2.363517e+11,1.356425e+11,1.543936e+11,35672735,194.372821,126.747093,220.851841,144.268519
3,2014-09-15,1.926042e+11,2.155333e+11,1.256883e+11,1.407813e+11,35707104,179.800090,117.332633,201.204835,131.422278
4,2014-10-15,1.778200e+11,1.908219e+11,1.160487e+11,1.248884e+11,35654873,166.241883,108.492559,178.397208,116.756722


In [57]:
statewide_monthly_agg.to_csv('statewide_monthly_agg.csv', index=False)

In [8]:
##### SUMMER CONSERVATION #####

# Extract rows of interest (summer months- June, July, August)
summer_months = ['2016-08-15', '2016-07-15', '2016-06-15', '2015-08-15', '2015-07-15', '2015-06-15', '2014-08-15', '2014-07-15', '2014-06-15']
summer_conservation_data = conservation_data.copy()
summer_conservation_data = summer_conservation_data[summer_conservation_data['date'].isin(summer_months)]

# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = summer_conservation_data.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
summer_conservation_data = summer_conservation_data[~summer_conservation_data["supplier_name"].isin(incomplete["supplier_name"])]

# Display first rows of table
summer_conservation_data.head()

,supplier_name,date,population,conservation_standard,production_month,production_2013,CII_production,gpcd,residential_use,hydrologic_region,water_days,complaints,res_production,conservation_month_gal,year,month
1,East Bay Municipal Utilities District,2016-08-15,1400000,0.00,6.007500e+09,7.172300e+09,1.141000e+09,84.437212,61.0,San Francisco Bay,7.0,176,3.664575e+09,1.164800e+09,2016,8
2,East Bay Municipal Utilities District,2016-07-15,1400000,0.00,6.056600e+09,7.452200e+09,9.940000e+08,83.731797,60.0,San Francisco Bay,7.0,197,3.633960e+09,1.395600e+09,2016,7
3,East Bay Municipal Utilities District,2016-06-15,1400000,0.00,5.675900e+09,6.927500e+09,8.390000e+08,82.435690,61.0,San Francisco Bay,2.0,188,3.577709e+09,1.251600e+09,2016,6
13,East Bay Municipal Utilities District,2015-08-15,1390000,0.16,5.250500e+09,7.172300e+09,9.972000e+08,74.328266,61.0,San Francisco Bay,2.0,653,3.202805e+09,1.921800e+09,2015,8
14,East Bay Municipal Utilities District,2015-07-15,1390000,0.16,5.148500e+09,7.452200e+09,9.550000e+08,71.689487,60.0,San Francisco Bay,2.0,740,3.089100e+09,2.303700e+09,2015,7


In [ ]:
summer_conservation_data.info()

In [ ]:
# Save as csv
summer_conservation_data.to_csv('summer_conservation.csv', index=False)

In [11]:
##### AGGREGATING SUMMER CONSERVATION DATA #####

##### 2016 #####
# Extract each year for summer-year calculations
summer_2016 = ['2016-08-15','2016-07-15','2016-06-15']
summer_2016_data = summer_conservation_data.copy()
summer_2016_data = summer_2016_data[summer_2016_data['date'].isin(summer_2016)]
summer_2016_data['residential'] = summer_2016_data['production_month']*summer_2016_data['residential_use']/100

# General values for population and hydrologic region 
latest = ['2015-06-15']
latest_data = summer_conservation_data.copy()
latest_data = latest_data[latest_data['date'].isin(latest)]
general_data = latest_data.loc[1:,['population', 'hydrologic_region']]
population = pd.DataFrame({'population' : summer_2016_data.groupby('hydrologic_region').sum()['population']}).reset_index()

residential_2016 = pd.DataFrame({'residential_2016' : summer_2016_data.groupby('hydrologic_region').sum()['residential']}).reset_index()


##### 2015 #####
# Extract each year for summer-year calculations
summer_2015 = ['2015-08-15','2015-07-15','2015-06-15']
summer_2015_data = summer_conservation_data.copy()
summer_2015_data = summer_2015_data[summer_2015_data['date'].isin(summer_2015)]
summer_2015_data['residential'] = summer_2015_data['production_month']*summer_2015_data['residential_use']/100

residential_2015 = pd.DataFrame({'residential_2015' : summer_2015_data.groupby('hydrologic_region').sum()['residential']}).reset_index()


##### 2014 #####
# Extract each year for summer-year calculations
summer_2014 = ['2014-08-15','2014-07-15','2014-06-15']
summer_2014_data = summer_conservation_data.copy()
summer_2014_data = summer_2014_data[summer_2014_data['date'].isin(summer_2014)]
summer_2014_data['residential'] = summer_2014_data['production_month']*summer_2014_data['residential_use']/100
summer_2014_data['residential2013'] = summer_2014_data['production_2013']*summer_2014_data['residential_use']/100

residential_2014 = pd.DataFrame({'residential_2014' : summer_2014_data.groupby('hydrologic_region').sum()['residential']}).reset_index()
residential_2013 = pd.DataFrame({'residential_2013' : summer_2014_data.groupby('hydrologic_region').sum()['residential2013']}).reset_index()

##### Merge 2014, 2015, 2016 data frames #####
summer_conservation_by_region = pd.merge(residential_2016, residential_2015, on='hydrologic_region',how='outer')
summer_conservation_by_region = pd.merge(summer_conservation_by_region, residential_2014, on='hydrologic_region',how='outer')
summer_conservation_by_region = pd.merge(summer_conservation_by_region, residential_2013, on='hydrologic_region',how='outer')
summer_conservation_by_region = pd.merge(summer_conservation_by_region, population, on='hydrologic_region',how='outer')

summer_conservation_by_region['rgpcd2016'] = summer_conservation_by_region['residential_2016']/summer_conservation_by_region['population']/30
summer_conservation_by_region['rgpcd2015'] = summer_conservation_by_region['residential_2015']/summer_conservation_by_region['population']/30
summer_conservation_by_region['rgpcd2014'] = summer_conservation_by_region['residential_2014']/summer_conservation_by_region['population']/30
summer_conservation_by_region['rgpcd2013'] = summer_conservation_by_region['residential_2013']/summer_conservation_by_region['population']/30

summer_conservation_by_region.head(10)

,hydrologic_region,residential_2016,residential_2015,residential_2014,residential_2013,population,rgpcd2016,rgpcd2015,rgpcd2014,rgpcd2013
0,Central Coast,8.664214e+09,8.132940e+09,1.031003e+10,1.179714e+10,3615206,79.886772,74.988262,95.061740,108.773328
1,Colorado River,1.268298e+10,1.152267e+10,1.536154e+10,1.629743e+10,2221712,190.288360,172.879726,230.476029,244.517641
2,North Coast,2.711857e+09,2.424698e+09,2.858284e+09,3.158083e+09,1085076,83.307757,74.486282,87.805962,97.015709
3,North Lahontan,9.862970e+08,8.095785e+08,1.063025e+09,1.150875e+09,258002,127.427563,104.595891,137.340613,148.690679
4,Sacramento River,4.271457e+10,3.498132e+10,4.394470e+10,5.403365e+10,7976067,178.511422,146.192834,183.652350,225.815778
5,San Francisco Bay,4.361105e+10,3.816020e+10,5.018179e+10,5.730531e+10,17530827,82.922602,72.558287,95.416277,108.961036
6,San Joaquin River,2.005999e+10,1.749984e+10,2.500326e+10,2.806084e+10,4416330,151.407728,132.084337,188.718211,211.796089
7,South Coast,1.803105e+11,1.640687e+11,2.130589e+11,2.212359e+11,59455243,101.090305,91.984421,119.450571,124.034976
8,South Lahontan,1.154661e+10,1.041358e+10,1.380666e+10,1.482043e+10,2456523,156.679645,141.305102,187.346979,201.103136
9,Tulare Lake,3.072044e+10,2.673618e+10,3.264493e+10,3.595030e+10,5513594,185.725421,161.637956,197.360246,217.343413


In [21]:
##### AGGREGATING SUMMER CONSERVATION BY REGION #####

#Aggregate conservation in summer months, re-build data frame for each year

# General values for population and hydrologic region 
latest = ['2015-06-15']
latest_data = summer_conservation_data.copy()
latest_data = latest_data[latest_data['date'].isin(latest)]
general_data = latest_data.loc[1:,['supplier_name', 'hydrologic_region', 'population', 'conservation_standard']]

## 2016 ##
conservation_16 = pd.DataFrame({'conservation_month_16' : summer_2016_data.groupby('supplier_name').sum()['conservation_month_gal']}).reset_index()
production_month_16 = pd.DataFrame({'production_month_16' : summer_2016_data.groupby('supplier_name').sum()['production_month']}).reset_index()
production_2013_16 = pd.DataFrame({'production_2013_16' : summer_2016_data.groupby('supplier_name').sum()['production_2013']}).reset_index()
CII_production_16 = pd.DataFrame({'CII_production_16' : summer_2016_data.groupby('supplier_name').sum()['CII_production']}).reset_index()
gpcd_16 = pd.DataFrame({'gpcd_16' : summer_2016_data.groupby('supplier_name').mean()['gpcd']}).reset_index()
residential_use_16 = pd.DataFrame({'residential_use_16' : summer_2016_data.groupby('supplier_name').mean()['residential_use']}).reset_index()
complaints_16 = pd.DataFrame({'complaints_16' : summer_2016_data.groupby('supplier_name').sum()['complaints']}).reset_index()
res_production_16 = pd.DataFrame({'res_production_16' : summer_2016_data.groupby('supplier_name').sum()['res_production']}).reset_index()

conservation_2016 = pd.merge(conservation_16, production_month_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, production_2013_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, CII_production_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, gpcd_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, residential_use_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, complaints_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, res_production_16, on='supplier_name', how='outer')
conservation_2016 = pd.merge(conservation_2016, general_data, on='supplier_name', how='outer')

## 2015 ##
conservation_15 = pd.DataFrame({'conservation_month_15' : summer_2015_data.groupby('supplier_name').sum()['conservation_month_gal']}).reset_index()
production_month_15 = pd.DataFrame({'production_month_15' : summer_2015_data.groupby('supplier_name').sum()['production_month']}).reset_index()
production_2013_15 = pd.DataFrame({'production_2013_15' : summer_2015_data.groupby('supplier_name').sum()['production_2013']}).reset_index()
CII_production_15 = pd.DataFrame({'CII_production_15' : summer_2015_data.groupby('supplier_name').sum()['CII_production']}).reset_index()
gpcd_15 = pd.DataFrame({'gpcd_15' : summer_2015_data.groupby('supplier_name').mean()['gpcd']}).reset_index()
residential_use_15 = pd.DataFrame({'residential_use_15' : summer_2015_data.groupby('supplier_name').mean()['residential_use']}).reset_index()
complaints_15 = pd.DataFrame({'complaints_15' : summer_2015_data.groupby('supplier_name').sum()['complaints']}).reset_index()
res_production_15 = pd.DataFrame({'res_production_15' : summer_2015_data.groupby('supplier_name').sum()['res_production']}).reset_index()

conservation_2015 = pd.merge(conservation_15, production_month_15, on='supplier_name', how='outer')
conservation_2015 = pd.merge(conservation_2015, production_2013_15, on='supplier_name', how='outer')
conservation_2015 = pd.merge(conservation_2015, CII_production_15, on='supplier_name', how='outer')
conservation_2015 = pd.merge(conservation_2015, gpcd_15, on='supplier_name', how='outer')
conservation_2015 = pd.merge(conservation_2015, residential_use_15, on='supplier_name', how='outer')
conservation_2015 = pd.merge(conservation_2015, complaints_15, on='supplier_name', how='outer')
conservation_2015 = pd.merge(conservation_2015, res_production_15, on='supplier_name', how='outer')

## 2014 ##
conservation_14 = pd.DataFrame({'conservation_month_14' : summer_2014_data.groupby('supplier_name').sum()['conservation_month_gal']}).reset_index()
production_month_14 = pd.DataFrame({'production_month_14' : summer_2014_data.groupby('supplier_name').sum()['production_month']}).reset_index()
production_2013_14 = pd.DataFrame({'production_2013_14' : summer_2014_data.groupby('supplier_name').sum()['production_2013']}).reset_index()
CII_production_14 = pd.DataFrame({'CII_production_14' : summer_2014_data.groupby('supplier_name').sum()['CII_production']}).reset_index()
gpcd_14 = pd.DataFrame({'gpcd_14' : summer_2016_data.groupby('supplier_name').mean()['gpcd']}).reset_index()
residential_use_14 = pd.DataFrame({'residential_use_14' : summer_2014_data.groupby('supplier_name').mean()['residential_use']}).reset_index()
complaints_14 = pd.DataFrame({'complaints_14' : summer_2014_data.groupby('supplier_name').sum()['complaints']}).reset_index()
res_production_14 = pd.DataFrame({'res_production_14' : summer_2014_data.groupby('supplier_name').sum()['res_production']}).reset_index()

conservation_2014 = pd.merge(conservation_14, production_month_14, on='supplier_name', how='outer')
conservation_2014 = pd.merge(conservation_2014, production_2013_14, on='supplier_name', how='outer')
conservation_2014 = pd.merge(conservation_2014, CII_production_14, on='supplier_name', how='outer')
conservation_2014 = pd.merge(conservation_2014, gpcd_14, on='supplier_name', how='outer')
conservation_2014 = pd.merge(conservation_2014, residential_use_14, on='supplier_name', how='outer')
conservation_2014 = pd.merge(conservation_2014, complaints_14, on='supplier_name', how='outer')
conservation_2014 = pd.merge(conservation_2014, res_production_14, on='supplier_name', how='outer')

## Merge data frames ##
summer_conservation = pd.merge(conservation_2014, conservation_2015, on='supplier_name', how='outer')
summer_conservation = pd.merge(summer_conservation, conservation_2016, on='supplier_name', how='outer')

summer_conservation.head()

,supplier_name,conservation_month_14,production_month_14,production_2013_14,CII_production_14,gpcd_14,residential_use_14,complaints_14,res_production_14,conservation_month_15,...,production_month_16,production_2013_16,CII_production_16,gpcd_16,residential_use_16,complaints_16,res_production_16,hydrologic_region,population,conservation_standard
0,Adelanto City of,1.683715e+07,4.315258e+08,4.483630e+08,0.000000e+00,117.483404,73.333333,0,3.218515e+08,6.083190e+07,...,4.340242e+08,3.933422e+08,6.914693e+07,117.483404,72.543333,3,3.181497e+08,South Lahontan,28723,0.20
1,Alco Water Service,4.041100e+07,4.075720e+08,4.479830e+08,0.000000e+00,103.254655,84.233333,0,3.472183e+08,9.708400e+07,...,3.442990e+08,4.479830e+08,6.255400e+07,103.254655,81.383333,7,2.833070e+08,Central Coast,29179,0.24
2,Alhambra City of,6.526478e+07,9.954500e+08,1.060715e+09,0.000000e+00,89.178967,100.000000,0,1.006558e+09,2.827087e+08,...,7.756372e+08,1.060725e+09,0.000000e+00,89.178967,90.000000,73,7.055237e+08,South Coast,85068,0.24
3,Amador Water Agency,6.081000e+07,3.771000e+08,4.379100e+08,0.000000e+00,102.606426,68.333333,0,2.612583e+08,1.437700e+08,...,3.509100e+08,4.312200e+08,1.033000e+08,102.606426,62.793333,1,2.227864e+08,San Joaquin River,23347,0.24
4,"American Canyon, City of",5.311378e+07,3.346494e+08,3.877632e+08,1.300147e+08,76.211112,53.666667,0,1.813969e+08,1.055759e+08,...,2.795805e+08,3.877632e+08,1.231718e+08,76.211112,51.000000,8,1.439853e+08,San Francisco Bay,20080,0.20


In [22]:
##### Compute values of interest on aggregated summer data #####

# Conservation wrt 2013 (%)
summer_conservation['conservation_percent_14'] = summer_conservation['conservation_month_14']/summer_conservation['production_2013_14']*100
summer_conservation['conservation_percent_15'] = summer_conservation['conservation_month_15']/summer_conservation['production_2013_15']*100
summer_conservation['conservation_percent_16'] = summer_conservation['conservation_month_16']/summer_conservation['production_2013_16']*100

# Percentage point difference from 2015 conservation (%)
summer_conservation['conservation_diff_14'] = summer_conservation['conservation_percent_14'] - summer_conservation['conservation_percent_15']
summer_conservation['conservation_diff_15'] = summer_conservation['conservation_percent_15'] - summer_conservation['conservation_percent_15']
summer_conservation['conservation_diff_16'] = summer_conservation['conservation_percent_16'] - summer_conservation['conservation_percent_15']

# Conservation backslide in 2016 (% difference from 2015 water use)
summer_conservation['backslide'] = (summer_conservation['production_month_16'] - summer_conservation['production_month_15'])/summer_conservation['production_month_15']*100
summer_conservation['backslide'] = summer_conservation['backslide'].round(2)

# Convert fraction to %
summer_conservation['conservation_standard'] = summer_conservation['conservation_standard']*100

# Conservation standard (%)
# Complaints (#)
# GPCD (gross)
# Percent residential use (%)
# Population
# Hydrologic region

##### Extract columns of interest #####
summer_conservation_df = summer_conservation.loc[1:,[
        'supplier_name', 
        'hydrologic_region', 
        'population', 
        'conservation_standard',
        'conservation_percent_14',
        'conservation_percent_15',
        'conservation_percent_16',
        'conservation_diff_16',
        'backslide',
        'complaints_14',
        'complaints_15',
        'complaints_16',
        'gpcd_14',
        'gpcd_15',
        'gpcd_16',
    ]]

summer_conservation_df.head()

,supplier_name,hydrologic_region,population,conservation_standard,conservation_percent_14,conservation_percent_15,conservation_percent_16,conservation_diff_16,backslide,complaints_14,complaints_15,complaints_16,gpcd_14,gpcd_15,gpcd_16
1,Alco Water Service,Central Coast,29179,24.0,9.020655,21.671358,23.144628,1.473270,-1.88,0,17,7,103.254655,109.453085,103.254655
2,Alhambra City of,South Coast,85068,24.0,6.152906,26.652904,26.876667,0.223763,-0.30,0,143,73,89.178967,99.392656,89.178967
3,Amador Water Agency,San Joaquin River,23347,24.0,13.886415,33.337971,18.623904,-14.714067,22.06,0,31,1,102.606426,82.002518,102.606426
4,"American Canyon, City of",San Francisco Bay,20080,20.0,13.697479,27.226891,27.899160,0.672269,-0.92,0,111,8,76.211112,76.865601,76.211112
5,Anaheim City of,South Coast,360142,20.0,0.465022,26.486106,15.895605,-10.590501,14.32,0,2609,636,92.560242,83.613746,92.560242


In [23]:
summer_conservation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 376 entries, 1 to 376
Data columns (total 15 columns):
supplier_name              376 non-null object
hydrologic_region          376 non-null object
population                 376 non-null int64
conservation_standard      376 non-null float64
conservation_percent_14    376 non-null float64
conservation_percent_15    376 non-null float64
conservation_percent_16    376 non-null float64
conservation_diff_16       376 non-null float64
backslide                  376 non-null float64
complaints_14              376 non-null int64
complaints_15              376 non-null int64
complaints_16              376 non-null int64
gpcd_14                    376 non-null float64
gpcd_15                    376 non-null float64
gpcd_16                    376 non-null float64
dtypes: float64(9), int64(4), object(2)
memory usage: 47.0+ KB


In [ ]:
# Save as csv
summer_conservation_df.to_csv('summer_conservation_agg.csv', index=False)

In [24]:
# For comparison graph:
summer_comparison = summer_conservation.loc[1:,[
        'supplier_name', 
        'hydrologic_region', 
        'conservation_standard', 
        'conservation_percent_15',
        'conservation_percent_16'
    ]]

# Save as csv
summer_comparison.to_csv('summer_comparison.csv', index=False)

In [47]:
##### BAY AREA SUMMER CONSERVATION #####

# Extract Bay Area suppliers
ba_conservation = summer_conservation_data.copy()
ba_conservation = ba_conservation[ba_conservation["hydrologic_region"] == "San Francisco Bay"]

# 2015 mandate
ba_conservation['mandate'] = ba_conservation['conservation_standard']*100

# Compute conservation % each month (w.r.t 2013)
ba_conservation['conservation'] = ba_conservation['conservation_month_gal']/ba_conservation['production_2013']*100

# Compute rebound in summer 2016 (% point difference with summer 2015)
ba_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
ba_rebound = ba_rebound[ba_rebound["hydrologic_region"] == "San Francisco Bay"]
ba_rebound = ba_rebound.loc[1:,['supplier_name', 'backslide']]

# Merge and extract columns of interest
sfbay_conservation = pd.merge(ba_conservation, ba_rebound, on='supplier_name', how='outer')
sfbay_conservation = sfbay_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]


# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = sfbay_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
sfbay_conservation = sfbay_conservation[~sfbay_conservation["supplier_name"].isin(incomplete["supplier_name"])]


sfbay_conservation.head()

,supplier_name,date,mandate,conservation_month_gal,production_2013,backslide,water_days
9,Coastside County Water District,2016-08-15,0.0,15590000.0,84560000.0,12.05,7.0
10,Coastside County Water District,2016-07-15,0.0,14370000.0,75610000.0,12.05,7.0
11,Coastside County Water District,2016-06-15,0.0,2320000.0,69550000.0,12.05,2.0
12,Coastside County Water District,2015-08-15,8.0,22560000.0,84560000.0,12.05,2.0
13,Coastside County Water District,2015-07-15,8.0,18280000.0,75610000.0,12.05,2.0


In [39]:
sfbay_conservation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 9 to 386
Data columns (total 7 columns):
supplier_name             378 non-null object
date                      378 non-null object
mandate                   378 non-null float64
conservation_month_gal    378 non-null float64
production_2013           378 non-null float64
backslide                 378 non-null float64
water_days                378 non-null float64
dtypes: float64(5), object(2)
memory usage: 23.6+ KB


In [ ]:
# Save as csv
sfbay_conservation.to_csv('sfbay_conservation.csv', index=False)

In [48]:
##### OTHER REGIONS #####

## NORTH COAST ##
# Extract North Coast suppliers
nc_conservation = summer_conservation_data.copy()
nc_conservation = nc_conservation[nc_conservation["hydrologic_region"] == "North Coast"]
# 2015 mandate
nc_conservation['mandate'] = nc_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
nc_conservation['conservation'] = nc_conservation['conservation_month_gal']/nc_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
nc_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
nc_rebound = nc_rebound[nc_rebound["hydrologic_region"] == "North Coast"]
nc_rebound = nc_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
nc_conservation = pd.merge(nc_conservation, nc_rebound, on='supplier_name', how='outer')
nc_conservation = nc_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = nc_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
nc_conservation = nc_conservation[~nc_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## CENTRAL COAST ##
# Extract Central Coast suppliers
cc_conservation = summer_conservation_data.copy()
cc_conservation = cc_conservation[cc_conservation["hydrologic_region"] == "Central Coast"]
# 2015 mandate
cc_conservation['mandate'] = cc_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
cc_conservation['conservation'] = cc_conservation['conservation_month_gal']/cc_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
cc_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
cc_rebound = cc_rebound[cc_rebound["hydrologic_region"] == "Central Coast"]
cc_rebound = cc_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
cc_conservation = pd.merge(cc_conservation, cc_rebound, on='supplier_name', how='outer')
cc_conservation = cc_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = cc_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
cc_conservation = cc_conservation[~cc_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## SOUTH COAST ##
# Extract South Coast suppliers
sc_conservation = summer_conservation_data.copy()
sc_conservation = sc_conservation[sc_conservation["hydrologic_region"] == "South Coast"]
# 2015 mandate
sc_conservation['mandate'] = sc_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
sc_conservation['conservation'] = sc_conservation['conservation_month_gal']/sc_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
sc_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
sc_rebound = sc_rebound[sc_rebound["hydrologic_region"] == "South Coast"]
sc_rebound = sc_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
sc_conservation = pd.merge(sc_conservation, sc_rebound, on='supplier_name', how='outer')
sc_conservation = sc_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = sc_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
sc_conservation = sc_conservation[~sc_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## SACRAMENTO RIVER ##
# Extract Sacramento River suppliers
sr_conservation = summer_conservation_data.copy()
sr_conservation = sr_conservation[sr_conservation["hydrologic_region"] == "Sacramento River"]
# 2015 mandate
sr_conservation['mandate'] = sr_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
sr_conservation['conservation'] = sr_conservation['conservation_month_gal']/sr_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
sr_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
sr_rebound = sr_rebound[sr_rebound["hydrologic_region"] == "Sacramento River"]
sr_rebound = sr_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
sr_conservation = pd.merge(sr_conservation, sr_rebound, on='supplier_name', how='outer')
sr_conservation = sr_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = sr_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
sr_conservation = sr_conservation[~sr_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## SAN JOAQUIN RIVER ##
# Extract San Joaquin River suppliers
sjc_conservation = summer_conservation_data.copy()
sjc_conservation = sjc_conservation[sjc_conservation["hydrologic_region"] == "San Joaquin River"]
# 2015 mandate
sjc_conservation['mandate'] = sjc_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
sjc_conservation['conservation'] = sjc_conservation['conservation_month_gal']/sjc_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
sjc_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
sjc_rebound = sjc_rebound[sjc_rebound["hydrologic_region"] == "San Joaquin River"]
sjc_rebound = sjc_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
sjc_conservation = pd.merge(sjc_conservation, sjc_rebound, on='supplier_name', how='outer')
sjc_conservation = sjc_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = sjc_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
sjc_conservation = sjc_conservation[~sjc_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## TULARE LAKE ##
# Extract Tulare Lake suppliers
tl_conservation = summer_conservation_data.copy()
tl_conservation = tl_conservation[tl_conservation["hydrologic_region"] == "Tulare Lake"]
# 2015 mandate
tl_conservation['mandate'] = tl_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
tl_conservation['conservation'] = tl_conservation['conservation_month_gal']/tl_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
tl_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
tl_rebound = tl_rebound[tl_rebound["hydrologic_region"] == "Tulare Lake"]
tl_rebound = tl_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
tl_conservation = pd.merge(tl_conservation, tl_rebound, on='supplier_name', how='outer')
tl_conservation = tl_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = tl_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
tl_conservation = tl_conservation[~tl_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## NORTH LAHONTAN ##
# Extract North Lahontan suppliers
nl_conservation = summer_conservation_data.copy()
nl_conservation = nl_conservation[nl_conservation["hydrologic_region"] == "North Lahontan"]
# 2015 mandate
nl_conservation['mandate'] = nl_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
nl_conservation['conservation'] = nl_conservation['conservation_month_gal']/nl_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
nl_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
nl_rebound = nl_rebound[nl_rebound["hydrologic_region"] == "North Lahontan"]
nl_rebound = nl_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
nl_conservation = pd.merge(nl_conservation, nl_rebound, on='supplier_name', how='outer')
nl_conservation = nl_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = nl_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
nl_conservation = nl_conservation[~nl_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## SOUTH LAHONTAN ##
# Extract South Lahontan suppliers
sl_conservation = summer_conservation_data.copy()
sl_conservation = sl_conservation[sl_conservation["hydrologic_region"] == "South Lahontan"]
# 2015 mandate
sl_conservation['mandate'] = sl_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
sl_conservation['conservation'] = sl_conservation['conservation_month_gal']/sl_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
sl_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
sl_rebound = sl_rebound[sl_rebound["hydrologic_region"] == "South Lahontan"]
sl_rebound = sl_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
sl_conservation = pd.merge(sl_conservation, sl_rebound, on='supplier_name', how='outer')
sl_conservation = sl_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = sl_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
sl_conservation = sl_conservation[~sl_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## COLORADO RIVER ##
# Extract Colorado River suppliers
cr_conservation = summer_conservation_data.copy()
cr_conservation = cr_conservation[cr_conservation["hydrologic_region"] == "Colorado River"]
# 2015 mandate
cr_conservation['mandate'] = cr_conservation['conservation_standard']*100
# Compute conservation % each month (w.r.t 2013)
cr_conservation['conservation'] = cr_conservation['conservation_month_gal']/cr_conservation['production_2013']*100
# Compute rebound in summer 2016 (% point difference with summer 2015)
cr_rebound = summer_conservation_df.loc[1:,['supplier_name', 'hydrologic_region', 'backslide']]
cr_rebound = cr_rebound[cr_rebound["hydrologic_region"] == "Colorado River"]
cr_rebound = cr_rebound.loc[1:,['supplier_name', 'backslide']]
# Merge and extract columns of interest
cr_conservation = pd.merge(cr_conservation, cr_rebound, on='supplier_name', how='outer')
cr_conservation = cr_conservation.loc[1:,[
        'supplier_name',
        'date',
        'mandate',
        'conservation_month_gal',
        'production_2013',
        'backslide',
        'water_days'
    ]]
# Remove suppliers with incomplete data (less than 9 months of data)
supplier_count = cr_conservation.groupby("supplier_name")["supplier_name"].count().to_frame("count").reset_index()
incomplete = supplier_count[supplier_count['count']<>9]
# Re-compile dataframe
cr_conservation = cr_conservation[~cr_conservation["supplier_name"].isin(incomplete["supplier_name"])]


## SAVE CSV FILES ##
nc_conservation.to_csv('nc_conservation.csv', index=False)
cc_conservation.to_csv('cc_conservation.csv', index=False)
sc_conservation.to_csv('sc_conservation.csv', index=False)
sr_conservation.to_csv('sr_conservation.csv', index=False)
sjc_conservation.to_csv('sjc_conservation.csv', index=False)
tl_conservation.to_csv('tl_conservation.csv', index=False)
nl_conservation.to_csv('nl_conservation.csv', index=False)
sl_conservation.to_csv('sl_conservation.csv', index=False)
cr_conservation.to_csv('cr_conservation.csv', index=False)